<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 2장: 텍스트 데이터 작업하기 (Working with Text Data)

## 이 장에서 다루는 내용

- 대규모 언어 모델 훈련을 위한 텍스트 준비
- 텍스트를 단어와 하위 단어 토큰으로 분할
- 텍스트 토큰화의 더 발전된 방법인 바이트 페어 인코딩
- 슬라이딩 윈도우 접근 방식으로 훈련 예제 샘플링
- 토큰을 대규모 언어 모델에 입력되는 벡터로 변환

이 노트북에서 사용되는 패키지들:

In [1]:
# 사용할 라이브러리의 버전 확인
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.8.0
tiktoken version: 0.11.0


- 이 장에서는 LLM을 위한 입력 데이터를 "준비"하는 데이터 준비 및 샘플링을 다룹니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

그림: LLM 데이터 준비의 전체 프로세스

## 2.1 단어 임베딩 이해하기

LLM을 포함한 심층 신경망 모델은 원시 텍스트를 직접 처리할 수 없습니다. 텍스트는 범주형이므로 신경망을 구현하고 훈련하는 데 사용되는 수학적 연산과 호환되지 않습니다. 따라서 단어를 연속값 벡터로 표현하는 방법이 필요합니다.

- 이 섹션에는 코드가 없습니다 - 개념 설명만 있습니다

- 임베딩의 다양한 형태가 있습니다; 이 책에서는 텍스트 임베딩에 초점을 맞춥니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

그림: 다양한 데이터 형태의 임베딩 예시

- LLM은 고차원 공간에서 임베딩으로 작업합니다 (즉, 수천 차원)
- 우리는 그런 고차원 공간을 시각화할 수 없으므로 (인간은 1, 2, 또는 3차원으로 생각함), 아래 그림은 2차원 임베딩 공간을 보여줍니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

그림: 2차원 임베딩 공간에서의 단어 관계

## 2.2 텍스트 토큰화

이 섹션에서는 텍스트를 토큰화합니다. 즉, 텍스트를 개별 단어나 구두점 문자와 같은 더 작은 단위로 나눕니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

그림: 텍스트 토큰화 과정

- 우리가 작업할 원시 텍스트를 로드합니다
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict)은 공개 도메인 단편 소설입니다

In [2]:
# 텍스트 파일 다운로드 (파일이 없는 경우)
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

<br>

---

<br>

#### SSL 인증서 오류 해결방법

- 일부 독자들이 VSCode나 Jupyter에서 `urllib.request.urlretrieve`를 실행할 때 ssl.SSLCertVerificationError: `SSL: CERTIFICATE_VERIFY_FAILED` 오류를 보고했습니다.
- 이는 보통 Python의 인증서 번들이 오래된 것을 의미합니다.

**해결 방법**

- Python ≥ 3.9 사용; 다음 코드를 실행하여 Python 버전을 확인할 수 있습니다:
```python
import sys
print(sys.__version__)
```
- 인증서 번들 업그레이드:
  - pip: `pip install --upgrade certifi`
  - uv: `uv pip install --upgrade certifi`
- 업그레이드 후 Jupyter 커널을 재시작하세요.
- 여전히 `ssl.SSLCertVerificationError`가 발생하면 [GitHub에서 자세한 정보](https://github.com/rasbt/LLMs-from-scratch/pull/403)를 참조하세요.

<br>

---

<br>

In [3]:
# 텍스트 파일 읽기
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("총 문자 수:", len(raw_text))
print(raw_text[:99])  # 처음 99자 출력

총 문자 수: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 목표는 이 텍스트를 LLM을 위해 토큰화하고 임베딩하는 것입니다
- 나중에 위의 텍스트에 적용할 수 있는 간단한 샘플 텍스트를 기반으로 한 간단한 토큰화기를 개발해보겠습니다
- 다음 정규 표현식은 공백으로 분할합니다

In [4]:
# 정규 표현식을 사용한 기본 토큰화
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)  # 공백으로 분할

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 우리는 공백뿐만 아니라 쉼표와 마침표로도 분할하고 싶으므로, 정규 표현식을 수정해보겠습니다

In [5]:
# 구두점을 포함한 분할
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 보시는 바와 같이 이것은 빈 문자열을 생성하므로 제거해보겠습니다

In [6]:
# 각 항목에서 공백을 제거하고 빈 문자열을 필터링합니다
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 이것은 꽤 좋아 보이지만, 마침표, 물음표 등과 같은 다른 유형의 구두점도 처리해보겠습니다

In [ ]:
# 다양한 구두점 처리
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

- 이제 꽤 좋아졌으므로 원시 텍스트에 이 토큰화를 적용할 준비가 되었습니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

그림: 토큰화 과정의 예시

In [ ]:
# 전체 텍스트에 토큰화 적용
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])  # 처음 30개 토큰 출력

- 총 토큰 수를 계산해보겠습니다

In [ ]:
print(len(preprocessed))

## 2.3 토큰을 토큰 ID로 변환하기

다음으로, 나중에 임베딩 레이어를 통해 처리할 수 있는 토큰 ID로 텍스트 토큰을 변환합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

그림: 토큰을 ID로 변환하는 과정

- 이러한 토큰들로부터 모든 고유한 토큰으로 구성된 어휘(vocabulary)를 구축할 수 있습니다

In [ ]:
# 어휘 구축
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

In [ ]:
# 토큰을 정수로 매핑하는 어휘 사전 생성
vocab = {token:integer for integer,token in enumerate(all_words)}

- 아래는 이 어휘의 처음 50개 항목입니다:

In [ ]:
# 어휘의 처음 50개 항목 출력
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

- 아래에서는 작은 어휘를 사용하여 짧은 샘플 텍스트의 토큰화를 보여줍니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

그림: 작은 어휘를 사용한 토큰화 예시

- 이제 모든 것을 토큰화기 클래스로 정리해보겠습니다

In [ ]:
# 간단한 토큰화기 클래스 구현
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        """텍스트를 토큰 ID 리스트로 인코딩"""
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        """토큰 ID 리스트를 텍스트로 디코딩"""
        text = " ".join([self.int_to_str[i] for i in ids])
        # 지정된 구두점 앞의 공백을 제거
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- `encode` 함수는 텍스트를 토큰 ID로 변환합니다
- `decode` 함수는 토큰 ID를 다시 텍스트로 변환합니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

그림: 토큰화기의 인코딩/디코딩 과정

- 토큰화기를 사용하여 텍스트를 정수로 인코딩(즉, 토큰화)할 수 있습니다
- 이 정수들은 나중에 LLM의 입력으로 임베딩될 수 있습니다

In [ ]:
# 토큰화기 테스트
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

- 정수를 다시 텍스트로 디코딩할 수 있습니다

In [ ]:
# 디코딩 테스트
print(tokenizer.decode(ids))

In [ ]:
# 인코딩 후 바로 디코딩 테스트 (원본과 비교)
print(tokenizer.decode(tokenizer.encode(text)))

## 2.4 특수 컨텍스트 토큰 추가하기

알 수 없는 단어를 위한 "특수" 토큰과 텍스트의 끝을 나타내는 토큰을 추가하는 것이 유용합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

그림: 특수 토큰들의 용도

일부 토큰화기는 LLM에게 추가적인 컨텍스트를 제공하기 위해 특수 토큰을 사용합니다:

- `[BOS]` (beginning of sequence): 텍스트의 시작을 표시
- `[EOS]` (end of sequence): 텍스트가 끝나는 곳을 표시 (보통 서로 관련 없는 여러 텍스트를 연결할 때 사용)
- `[PAD]` (padding): 배치 크기가 1보다 큰 경우 LLM을 훈련할 때 사용 (여러 텍스트를 같은 길이로 맞춤)
- `[UNK]`: 어휘에 포함되지 않은 단어를 나타냄

GPT-2는 복잡성을 줄이기 위해 위에서 언급한 토큰들이 필요하지 않고 `<|endoftext|>` 토큰만 사용합니다.
GPT-2는 어휘에 없는 단어에 대해 `<UNK>` 토큰을 사용하지 않고, 대신 바이트 페어 인코딩(BPE) 토큰화기를 사용합니다.

- 두 개의 독립적인 텍스트 소스 사이에 `<|endoftext|>` 토큰을 사용합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

그림: 텍스트 사이에 endoftext 토큰 사용

- 다음 텍스트를 토큰화하면 어떻게 되는지 살펴보겠습니다:

In [ ]:
# 어휘에 없는 단어가 포함된 텍스트 토큰화 시도
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

try:
    tokenizer.encode(text)
except KeyError as e:
    print(f"KeyError: {e} - 이 단어는 어휘에 없습니다!")

- 위의 결과는 "Hello"라는 단어가 어휘에 포함되어 있지 않기 때문에 오류를 생성합니다
- 이런 경우를 처리하기 위해 알 수 없는 단어를 나타내는 `"<|unk|>"`과 같은 특수 토큰을 어휘에 추가할 수 있습니다
- 이미 어휘를 확장하고 있으므로, GPT-2 훈련에서 텍스트의 끝을 나타내는데 사용되는 `"<|endoftext|>"`라는 또 다른 토큰을 추가해보겠습니다

In [ ]:
# 특수 토큰이 포함된 새로운 어휘 생성
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [ ]:
# 새 어휘 크기 확인
print(len(vocab.items()))

In [ ]:
# 마지막 5개 항목 확인 (특수 토큰들 포함)
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

- 새로운 `<unk>` 토큰을 언제 어떻게 사용하는지 알 수 있도록 토큰화기를 적절히 조정해야 합니다

In [ ]:
# 특수 토큰을 처리하는 개선된 토큰화기
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        """텍스트를 토큰 ID로 인코딩 (알 수 없는 단어는 <|unk|>로 처리)"""
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        """토큰 ID를 텍스트로 디코딩"""
        text = " ".join([self.int_to_str[i] for i in ids])
        # 지정된 구두점 앞의 공백을 제거
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

수정된 토큰화기로 텍스트를 토큰화해보겠습니다:

In [ ]:
# 개선된 토큰화기 테스트
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

In [ ]:
# 인코딩 테스트
print(tokenizer.encode(text))

In [ ]:
# 디코딩 테스트
print(tokenizer.decode(tokenizer.encode(text)))

## 2.5 바이트페어 인코딩 (BytePair Encoding)

- GPT-2는 토큰화기로 바이트페어 인코딩(BPE)을 사용합니다
- 이는 모델이 미리 정의된 어휘에 없는 단어들을 더 작은 하위 단어 단위나 개별 문자로 분해할 수 있게 합니다
- 예를 들어, GPT-2의 어휘에 "unfamiliarword"라는 단어가 없다면, 훈련된 BPE 병합에 따라 ["unfam", "iliar", "word"] 또는 다른 하위 단어 분해로 토큰화할 수 있습니다

- 원본 BPE 토큰화기는 여기서 찾을 수 있습니다: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 이 장에서는 OpenAI의 오픈소스 [tiktoken](https://github.com/openai/tiktoken) 라이브러리의 BPE 토큰화기를 사용합니다. 이는 계산 성능을 향상시키기 위해 핵심 알고리즘을 Rust로 구현했습니다

In [ ]:
# tiktoken 설치 (필요한 경우)
# !pip install tiktoken

In [ ]:
# tiktoken 라이브러리 가져오기 및 버전 확인
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

In [ ]:
# GPT-2 토큰화기 로드
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
# BPE 토큰화기 테스트
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

In [ ]:
# 디코딩 테스트
strings = tokenizer.decode(integers)

print(strings)

- BPE 토큰화기는 알 수 없는 단어를 하위 단어와 개별 문자로 분해합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

그림: BPE 토큰화의 하위 단어 분해 과정

## 2.6 슬라이딩 윈도우로 데이터 샘플링하기

- 우리는 LLM이 한 번에 한 단어씩 생성하도록 훈련하므로, 시퀀스의 다음 단어가 예측할 타겟을 나타내는 훈련 데이터를 준비해야 합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

그림: 다음 단어 예측을 위한 입력-타겟 쌍

In [ ]:
# 전체 텍스트를 토큰화
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

- 각 텍스트 청크에 대해 입력과 타겟이 필요합니다
- 모델이 다음 단어를 예측하기를 원하므로, 타겟은 한 위치만큼 오른쪽으로 이동된 입력입니다

In [ ]:
# 샘플 텍스트 청크 생성
enc_sample = enc_text[50:]

context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

- 하나씩, 예측은 다음과 같이 보일 것입니다:

In [ ]:
# 각 단계별 예측 과정 보기
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

In [ ]:
# 토큰 ID를 실제 단어로 변환하여 보기
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

- 어텐션 메커니즘을 다룬 후에 다음 장에서 다음 단어 예측을 처리할 것입니다
- 지금은 입력 데이터셋을 반복하고 하나씩 이동된 입력과 타겟을 반환하는 간단한 데이터 로더를 구현하겠습니다

- PyTorch 설치 및 가져오기 (설치 팁은 부록 A 참조)

In [ ]:
# PyTorch 가져오기 및 버전 확인
import torch
print("PyTorch version:", torch.__version__)

- +1씩 위치를 변경하는 슬라이딩 윈도우 접근법을 사용합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

그림: 슬라이딩 윈도우 데이터 샘플링

- 입력 텍스트 데이터셋에서 청크를 추출하는 데이터셋과 데이터로더를 생성합니다

In [ ]:
# GPT 데이터셋 클래스 구현
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # 전체 텍스트를 토큰화
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "토큰화된 입력의 수는 최소 max_length+1과 같아야 합니다"

        # 슬라이딩 윈도우를 사용하여 책을 max_length의 겹치는 시퀀스로 청크화
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
# 데이터로더 생성 함수
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # 토큰화기 초기화
    tokenizer = tiktoken.get_encoding("gpt2")

    # 데이터셋 생성
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 데이터로더 생성
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- 컨텍스트 크기가 4인 LLM에 대해 배치 크기 1로 데이터로더를 테스트해보겠습니다:

In [ ]:
# 텍스트 파일 읽기
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
# 데이터로더 테스트 (배치 크기 1, 컨텍스트 크기 4)
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

In [ ]:
# 두 번째 배치 확인
second_batch = next(data_iter)
print(second_batch)

- 아래와 같이 컨텍스트 길이(여기서는 4)와 동일한 스트라이드를 사용한 예시:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

그림: 스트라이드가 컨텍스트 길이와 같은 경우의 샘플링

- 배치된 출력도 생성할 수 있습니다
- 여기서 스트라이드를 증가시켜 배치 간에 겹치지 않도록 합니다. 더 많은 겹침은 과적합 증가로 이어질 수 있기 때문입니다

In [ ]:
# 배치 크기 8로 테스트
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("입력:")
print(inputs)
print("\n타겟:")
print(targets)

## 2.7 토큰 임베딩 생성하기

- 데이터는 이미 거의 LLM을 위해 준비되었습니다
- 마지막으로 임베딩 레이어를 사용하여 토큰을 연속 벡터 표현으로 임베딩해보겠습니다
- 일반적으로 이러한 임베딩 레이어는 LLM 자체의 일부이며 모델 훈련 중에 업데이트(훈련)됩니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

그림: 토큰 임베딩 과정

- (토큰화 후) 입력 ID 2, 3, 5, 1을 가진 다음 네 가지 입력 예시가 있다고 가정해보겠습니다:

In [ ]:
# 예시 입력 ID
input_ids = torch.tensor([2, 3, 5, 1])

- 간단함을 위해 6개 단어만의 작은 어휘와 크기 3의 임베딩을 생성한다고 가정해보겠습니다:

In [ ]:
# 임베딩 레이어 생성
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 이는 6x3 가중치 행렬을 생성합니다:

In [ ]:
# 임베딩 레이어의 가중치 확인
print(embedding_layer.weight)

- 원-핫 인코딩에 익숙한 분들을 위해, 위의 임베딩 레이어 접근법은 본질적으로 원-핫 인코딩 후 완전 연결 레이어에서 행렬 곱셈을 수행하는 것의 더 효율적인 구현일 뿐입니다
- 임베딩 레이어가 원-핫 인코딩 및 행렬 곱셈 접근법과 동등한 더 효율적인 구현이기 때문에, 역전파를 통해 최적화할 수 있는 신경망 레이어로 볼 수 있습니다

- ID 3인 토큰을 3차원 벡터로 변환하려면 다음과 같이 합니다:

In [ ]:
# 단일 토큰 임베딩
print(embedding_layer(torch.tensor([3])))

- 위의 결과는 `embedding_layer` 가중치 행렬의 4번째 행입니다
- 위의 네 `input_ids` 값을 모두 임베딩하려면:

In [ ]:
# 여러 토큰 임베딩
print(embedding_layer(input_ids))

- 임베딩 레이어는 본질적으로 룩업 연산입니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

그림: 임베딩 레이어의 룩업 과정

## 2.8 단어 위치 인코딩하기

- 임베딩 레이어는 입력 시퀀스에서 위치에 관계없이 ID를 동일한 벡터 표현으로 변환합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

그림: 위치와 무관한 토큰 임베딩

- 위치 임베딩은 토큰 임베딩 벡터와 결합되어 대규모 언어 모델의 입력 임베딩을 형성합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

그림: 토큰 임베딩과 위치 임베딩의 결합

- 바이트페어 인코더는 50,257개의 어휘 크기를 가집니다:
- 입력 토큰을 256차원 벡터 표현으로 인코딩한다고 가정해보겠습니다:

In [ ]:
# GPT-2 크기의 토큰 임베딩 레이어
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 데이터로더에서 데이터를 샘플링하면, 각 배치의 토큰을 256차원 벡터로 임베딩합니다
- 각각 4개의 토큰을 가진 배치 크기 8이 있다면, 이는 8 x 4 x 256 텐서가 됩니다:

In [ ]:
# 실제 데이터로 토큰 임베딩 테스트
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [ ]:
print("토큰 ID:")
print(inputs)
print("\n입력 크기:")
print(inputs.shape)

In [ ]:
# 토큰 임베딩 생성
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# 임베딩이 어떻게 보이는지 확인하려면 아래 줄의 주석을 제거하고 실행하세요
# print(token_embeddings)

- GPT-2는 절대 위치 임베딩을 사용하므로, 단순히 다른 임베딩 레이어를 생성합니다:

In [ ]:
# 위치 임베딩 레이어 생성
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# 임베딩 레이어 가중치를 보려면 아래 줄의 주석을 제거하고 실행하세요
# print(pos_embedding_layer.weight)

In [ ]:
# 위치 임베딩 생성
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# 임베딩이 어떻게 보이는지 확인하려면 아래 줄의 주석을 제거하고 실행하세요
# print(pos_embeddings)

- LLM에서 사용되는 입력 임베딩을 생성하려면, 토큰 임베딩과 위치 임베딩을 단순히 더합니다:

In [ ]:
# 최종 입력 임베딩 생성 (토큰 + 위치 임베딩)
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# 임베딩이 어떻게 보이는지 확인하려면 아래 줄의 주석을 제거하고 실행하세요
# print(input_embeddings)

- 입력 처리 워크플로우의 초기 단계에서, 입력 텍스트는 개별 토큰으로 분할됩니다
- 이 분할 후, 이러한 토큰은 미리 정의된 어휘를 기반으로 토큰 ID로 변환됩니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

그림: 전체 텍스트 처리 파이프라인

# 요약 및 핵심 사항

이 장에서 구현한 데이터 로더의 간결한 버전은 [./dataloader.ipynb](./dataloader.ipynb) 코드 노트북을 참조하세요. 이는 향후 장에서 GPT 모델을 훈련하는 데 필요할 것입니다.

연습 문제 해답은 [./exercise-solutions.ipynb](./exercise-solutions.ipynb)를 참조하세요.

GPT-2 토큰화기를 처음부터 구현하고 훈련하는 방법에 관심이 있다면 [Byte Pair Encoding (BPE) Tokenizer From Scratch](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) 노트북을 참조하세요.